In [1]:
# read the network details
def readNet(fileName):
    f = open(fileName, "r")
    net = {}
    n = int(f.readline())
    net['noNodes'] = n
    mat = []
    for i in range(n):
        mat.append([])
        line = f.readline()
        elems = line.split(" ")
        for j in range(n):
            mat[-1].append(int(elems[j]))
    net["mat"] = mat 
    degrees = []
    noEdges = 0
    for i in range(n):
        d = 0
        for j in range(n):
            if (mat[i][j] == 1):
                d += 1
            if (j > i):
                noEdges += mat[i][j]
        degrees.append(d)
    net["noEdges"] = noEdges
    net["degrees"] = degrees
    f.close()
    return net

In [2]:
import networkx as NX

def readGML(fileName):
    net={}
    # G=NX.read_gml('data/dolphins.gml')
    G=NX.read_gml(fileName)
    n=len(G.nodes())
    net['noNodes']=n
    mat=[]
    for i in range(n):
        mat.append([])
        for j in range(n):
            mat[i].append(0)
    sol = list(G.nodes())
    for element in G.edges():
        mat[sol.index(element[0])][sol.index(element[-1])]=1
    net['mat']=mat
    degrees = []
    noEdges = 0
    for i in range(n):
        d = 0
        for j in range(n):
            if (mat[i][j] == 1):
                d += 1
            if (j > i):
                noEdges += mat[i][j]
        degrees.append(d)
    net["noEdges"] = noEdges
    net["degrees"] = degrees
    return net
    # print(str(sol.index(element[0])) + " : " + str(sol.index(element[-1])))
    # print(G.nodes())
    # print(G.edges)

In [3]:
# define the function
import math
import numpy as np 

MIN = -5
MAX = 5

def fcEval(communities,param):
    noNodes = param['noNodes']
    mat = param['mat']
    degrees = param['degrees']
    noEdges = param['noEdges']  
    M = 2 * noEdges
    Q = 0.0
    for i in range(0, noNodes):
        for j in range(0, noNodes):
            if (communities[i] == communities[j]):
               Q += (mat[i][j] - degrees[i] * degrees[j] / M)
    return Q * 1 / M
 

**GA running**

Use the GA (with real encoding) for identify the optimal solution


In [4]:


from random import randint

class GA:
    def __init__(self, param = None, problParam = None):
        self.__param = param
        self.__problParam = problParam
        self.__population = []
        
    @property
    def population(self):
        return self.__population
    
    def initialisation(self):
        for _ in range(0, self.__param['popSize']):
            c = Chromosome(self.__problParam)
            self.__population.append(c)
    
    def evaluation(self):
        for c in self.__population:
            c.fitness = self.__problParam['function'](c.repres,net)
            
    def worstChromosome(self):
        best = self.__population[0]
        for c in self.__population:
            if (c.fitness < best.fitness):
                best = c
        return best
        
    def bestChromosome(self):
        best = self.__population[0]
        for c in self.__population:
            if (c.fitness > best.fitness):
                best = c
        return best

    def selection(self):
        pos1 = randint(0, self.__param['popSize'] - 1)
        pos2 = randint(0, self.__param['popSize'] - 1)
        if (self.__population[pos1].fitness > self.__population[pos2].fitness):
            return pos1
        else:
            return pos2 
        
    
    def oneGeneration(self):
        newPop = []
        for _ in range(self.__param['popSize']):
            p1 = self.__population[self.selection()]
            p2 = self.__population[self.selection()]
            off = p1.crossover(p2)
            off.mutation()
            newPop.append(off)
        self.__population = newPop
        self.evaluation()

    def oneGenerationElitism(self):
        newPop = [self.bestChromosome()]
        for _ in range(self.__param['popSize'] - 1):
            p1 = self.__population[self.selection()]
            p2 = self.__population[self.selection()]
            off = p1.crossover(p2)
            off.mutation()
            newPop.append(off)
        self.__population = newPop
        self.evaluation()
        
    def oneGenerationSteadyState(self):
        newPop = [self.bestChromosome()]
        for _ in range(self.__param['popSize']-1):
            p1 = self.__population[self.selection()]
            p2 = self.__population[self.selection()]
            off = p1.crossover(p2)
            off.mutation()
            off.fitness = self.__problParam['function'](off.repres)
            worst = self.worstChromosome()
            if (off.fitness < worst.fitness):
                worst = off        

In [5]:
def nrComunitati(repres):
    n=len(repres)
    aux=[0]*12
    nr=0
    for i in range(n):
        aux[repres[i]]=repres.count(repres[i])
    for x in aux:
        if x!=0:
            nr=nr+1
    return nr

In [6]:
from RealChromosome import Chromosome
#from BinChromosome import Chromosome
from random import seed 
import netconf

#seed(100)

#net=readNet("data/net.in")
net=readGML("data/karate.gml")
# print(net['mat'])
# print(net['degrees'])
MIN=1
if net['noNodes']<=11:
    MAX=net["noNodes"]
else:
    MAX=11
noDim=net["noNodes"]
# initialise de GA parameters
gaParam = {'popSize' : 100, 'noGen' : 100, 'pc' : 0.8, 'pm' : 0.1}
# problem parameters
problParam = {'min' : MIN, 'max' : MAX, 'function' : fcEval, 'noDim' : noDim, 'noBits' : 8}

# store the best/average solution of each iteration (for a final plot used to anlyse the GA's convergence)
allBestFitnesses = []
allAvgFitnesses = []
generations = []


ga = GA(gaParam, problParam)
ga.initialisation()
ga.evaluation()
maximFitness=-1
bestRepres=[]
fileName_output="karate_out.txt"
f=open(fileName_output,'w')
for g in range(gaParam['noGen']):
    #plotting preparation
    # allPotentialSolutionsX = [c.repres for c in ga.population]
    # allPotentialSolutionsY = [c.fitness for c in ga.population]
    bestSolX = ga.bestChromosome().repres
    bestSolY = ga.bestChromosome().fitness
    if bestSolY>maximFitness:
        maximFitness=bestSolY
        bestRepres=bestSolX
    allBestFitnesses.append(bestSolY)
    #allAvgFitnesses.append(sum(allPotentialSolutionsY) / len(allPotentialSolutionsY))
    #generations.append(g)
    

    #logic alg
    #ga.oneGeneration()
    ga.oneGenerationElitism()
    # ga.oneGenerationSteadyState()
    
    bestChromo = ga.bestChromosome()
    f.write('Best solution in generation ' + str(g) + ' is: x = ' + str(bestChromo.repres) + ' f(x) = ' + str(bestChromo.fitness))
    f.write('\n')
f.write("Best repres&fitness: "+str(bestRepres)+" "+str(maximFitness))
f.write('\n')
f.write("Nr. de comunitati: "+str(nrComunitati(bestRepres)))
f.write('\n')
f.write("Node - culoare comunitate: \n")
for i in range(len(bestRepres)):
    f.write(str(i)+" - "+str(bestRepres[i]))
    f.write('\n')
    
f.close()

NetworkXError: node #0 has no 'label' attribute